In [7]:
import cv2
import numpy as np
import json
from datetime import datetime, timedelta
import mediapipe as mp
from transformers import pipeline
import torch
import hashlib
import time

class AuraHealthAnalyzer:
    def __init__(self, use_huggingface=True):
        # Modern face detection
        self.face_detector = mp.solutions.face_detection.FaceDetection(
            model_selection=1, min_detection_confidence=0.7
        )
        
        self.face_mesh = mp.solutions.face_mesh.FaceMesh(
            static_image_mode=True,
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=0.7
        )
        self.celebrity_database = {}
        self._initialize_celebrity_database()
        self.analysis_history = []
        self.use_huggingface = use_huggingface
        
        self._load_knowledge_bases()
        
        if use_huggingface:
            self._initialize_huggingface_models()

    def _load_knowledge_bases(self):
        """Knowledge bases for intelligent recommendations"""
        self.skin_knowledge = {
            'dry_skin': {
                'diagnosis': 'Lack of lipids and moisture',
                'morning_routine': [
                    'Gentle cream-gel cleanser',
                    'Hyaluronic acid serum on damp skin',
                    'Cream with ceramides',
                    'SPF 30-50'
                ],
                'evening_routine': [
                    'Double cleansing',
                    'Peptide serum',
                    'Nourishing cream with squalane'
                ]
            },
            'oily_skin': {
                'diagnosis': 'Overactive sebaceous glands',
                'morning_routine': [
                    'Gel cleanser with salicylic acid',
                    'Niacinamide 10% serum',
                    'Oil-free moisturizing gel',
                    'Mattifying SPF 50'
                ]
            }
        }
    def _initialize_huggingface_models(self):
        """Initialize Hugging Face models for emotion detection"""
        try:
            self.emotion_detector = pipeline(
                "image-classification",
                model="trpakov/vit-face-expression",
                device=0 if torch.cuda.is_available() else -1
            )
            print("✅ Hugging Face models loaded successfully")
        except Exception as e:
            print(f"❌ Could not load Hugging Face models: {e}")
            self.use_huggingface = False
                
        self.energy_knowledge = {
            'mild_fatigue': {
                'recommendations': [
                    'Sleep 7-8 hours',
                    'Cold compresses in morning',
                    'Light cardio 20 minutes'
                ]
            },
            'chronic_fatigue': {
                'recommendations': [
                    'Consult physician',
                    'Blood test for iron, vitamin D',
                    'Meditation before sleep'
                ]
            }
        }
    def _initialize_celebrity_database(self):
        """Загружает базу знаменитостей из существующей папки"""
        import os
        import glob
        
        celeb_folder = "celebrities"
        
        if not os.path.exists(celeb_folder):
            print("❌ Папка celebrities не найдена!")
            return
        
        # Правильный поиск файлов с учетом кодировки
        image_files = []
        for ext in ["*.jpg", "*.png", "*.jpeg"]:
            image_files.extend(glob.glob(os.path.join(celeb_folder, ext)))
            image_files.extend(glob.glob(os.path.join(celeb_folder, ext.upper())))
        
        print(f"📁 Найдено {len(image_files)} фото знаменитостей")
        
        for image_path in image_files:
            try:
                # Проверяем что файл существует и читается
                if not os.path.exists(image_path):
                    print(f"❌ Файл не существует: {image_path}")
                    continue
                    
                if os.path.getsize(image_path) == 0:
                    print(f"❌ Пустой файл: {image_path}")
                    continue
                
                filename = os.path.basename(image_path)
                celeb_name = os.path.splitext(filename)[0]
                
                print(f"🔍 Анализируем: {celeb_name}")
                
                celeb_metrics = self._analyze_celebrity_photo(image_path)
                if celeb_metrics:
                    self.celebrity_database[celeb_name] = {
                        "name": celeb_name,
                        "image_path": image_path,
                        "metrics": celeb_metrics
                    }
                    print(f"✅ Проанализирована: {celeb_name}")
                else:
                    print(f"❌ Не удалось проанализировать: {celeb_name}")
                    
            except Exception as e:
                print(f"❌ Ошибка {os.path.basename(image_path)}: {str(e)}")
    def _initialize_celebrity_database(self):
        # Базовые метрики для разных типов знаменитостей
        self.celebrity_database = {
            "Криштиану Роналду": {
                "name": "Криштиану Роналду",
                "metrics": {
                    "skin_evenness": 0.9,
                    "skin_smoothness": 0.85,
                    "skin_cleanliness": 0.95,
                    "eye_freshness": 0.95,
                    "energy_level": 0.98
                }
            },
            "Марго Робби": {
                "name": "Марго Робби", 
                "metrics": {
                    "skin_evenness": 0.95,
                    "skin_smoothness": 0.92,
                    "skin_cleanliness": 0.90,
                    "eye_freshness": 0.88,
                    "energy_level": 0.85
                }
            },
            "Эмма Уотсон": {
                "name": "Эмма Уотсон",
                "metrics": {
                    "skin_evenness": 0.88,
                    "skin_smoothness": 0.90, 
                    "skin_cleanliness": 0.92,
                    "eye_freshness": 0.90,
                    "energy_level": 0.87
                }
            },
            "Брэд Питт": {
                "name": "Брэд Питт",
                "metrics": {
                    "skin_evenness": 0.86,
                    "skin_smoothness": 0.88,
                    "skin_cleanliness": 0.91,
                    "eye_freshness": 0.87,
                    "energy_level": 0.84
                }
            },
            "Скарлетт Йоханссон": {
                "name": "Скарлетт Йоханссон",
                "metrics": {
                    "skin_evenness": 0.91,
                    "skin_smoothness": 0.89,
                    "skin_cleanliness": 0.93,
                    "eye_freshness": 0.86,
                    "energy_level": 0.82
                }
            }
        }
    
    def _analyze_celebrity_photo(self, image_path):
        """Упрощенный анализ фото знаменитости"""
        try:
            # Пробуем просто загрузить изображение без сложной обработки
            img = cv2.imread(image_path)
            if img is None:
                print(f"❌ Не могу загрузить: {image_path}")
                return None
            
            # Базовая обработка
            img = cv2.resize(img, (500, 500))  # стандартный размер
            rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # Детекция лица
            results = self.face_detector.process(rgb_img)
            if not results.detections:
                print(f"❌ Не найдено лицо в: {image_path}")
                return None
            
            # Обрезаем лицо
            detection = results.detections[0]
            bbox = detection.location_data.relative_bounding_box
            h, w = img.shape[:2]
            
            x = int(bbox.xmin * w)
            y = int(bbox.ymin * h)
            face_w = int(bbox.width * w)
            face_h = int(bbox.height * h)
            
            # Добавляем margin
            margin = 0.1
            x = max(0, int(x - face_w * margin))
            y = max(0, int(y - face_h * margin))
            face_w = min(w - x, int(face_w * (1 + 2 * margin)))
            face_h = min(h - y, int(face_h * (1 + 2 * margin)))
            
            face_img = img[y:y+face_h, x:x+face_w]
            
            if face_img.size == 0:
                print(f"❌ Пустое лицо в: {image_path}")
                return None
            
            # Базовый анализ (без mesh для скорости)
            skin_quality = self._analyze_skin_quality(face_img)
            
            # Простой анализ энергии (без landmarks)
            eye_regions = self._extract_eye_regions_simple(face_img)
            eye_freshness = 0.7  # базовое значение для знаменитостей
            
            return {
                "skin_evenness": skin_quality["evenness"],
                "skin_smoothness": skin_quality["smoothness"], 
                "skin_cleanliness": skin_quality["cleanliness"],
                "eye_freshness": eye_freshness,
                "energy_level": 0.8  # базовый высокий уровень для знаменитостей
            }
            
        except Exception as e:
            print(f"❌ Ошибка анализа {image_path}: {str(e)}")
            return None

    def _extract_eye_regions_simple(self, face_img):
        """Упрощенное извлечение областей глаз"""
        # Для знаменитостей используем приблизительные области
        h, w = face_img.shape[:2]
        
        left_eye = face_img[int(h*0.2):int(h*0.45), int(w*0.1):int(w*0.4)]
        right_eye = face_img[int(h*0.2):int(h*0.45), int(w*0.6):int(w*0.9)]
        
        return [left_eye, right_eye] if left_eye.size > 0 and right_eye.size > 0 else []

    def _preprocess_image(self, image_input):
        """Универсальный препроцессинг: поддерживает пути, numpy arrays, bytes"""
        if isinstance(image_input, str):
            # Вход - путь к файлу (для тестирования)
            img = cv2.imread(image_input)
            if img is None:
                raise ValueError(f"Cannot load image from path: {image_input}")
        elif isinstance(image_input, np.ndarray):
            # Вход - numpy array (уже загружено)
            img = image_input
        elif isinstance(image_input, bytes):
            # Вход - bytes (БЭКЕНД)
            nparr = np.frombuffer(image_input, np.uint8)
            img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
            if img is None:
                raise ValueError("Cannot decode image from bytes")
        else:
            raise ValueError("Unsupported image input type")
        
        img = self._normalize_lighting(img)
        img = self._white_balance(img)
        img = self._denoise_image(img)
        face_img = self._detect_face_modern(img)
        
        return face_img

    def _normalize_lighting(self, img):
        """Lighting normalization using CLAHE"""
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        l_normalized = clahe.apply(l)
        lab_normalized = cv2.merge([l_normalized, a, b])
        return cv2.cvtColor(lab_normalized, cv2.COLOR_LAB2BGR)

    def _white_balance(self, img):
        """White balance correction"""
        result = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        avg_a = np.average(result[:, :, 1])
        avg_b = np.average(result[:, :, 2])
        result[:, :, 1] = result[:, :, 1] - ((avg_a - 128) * (result[:, :, 0] / 255.0) * 1.1)
        result[:, :, 2] = result[:, :, 2] - ((avg_b - 128) * (result[:, :, 0] / 255.0) * 1.1)
        return cv2.cvtColor(result, cv2.COLOR_LAB2BGR)

    def _denoise_image(self, img):
        """Image denoising"""
        return cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)

    def _detect_face_modern(self, img):
        """Modern face detection with MediaPipe"""
        rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = self.face_detector.process(rgb_img)
        
        if not results.detections:
            raise ValueError("No face detected")
        
        detection = results.detections[0]
        bbox = detection.location_data.relative_bounding_box
        h, w = img.shape[:2]
        
        x = int(bbox.xmin * w)
        y = int(bbox.ymin * h)
        face_w = int(bbox.width * w)
        face_h = int(bbox.height * h)
        
        margin = 0.1
        x = max(0, int(x - face_w * margin))
        y = max(0, int(y - face_h * margin))
        face_w = min(w - x, int(face_w * (1 + 2 * margin)))
        face_h = min(h - y, int(face_h * (1 + 2 * margin)))
        
        return img[y:y+face_h, x:x+face_w]

    def get_web_questionnaire(self):
        """Questionnaire for web interface"""
        return {
            "personal_info": {
                "age": {"type": "number", "label": "Age", "required": True},
                "gender": {"type": "select", "label": "Gender", "options": ["male", "female"], "required": True}
            },
            "main_concerns": {
                "primary": {
                    "type": "multiselect", 
                    "label": "Main Concerns", 
                    "options": [
                        "Fatigue and lack of sleep", 
                        "Dark circles under eyes",
                        "Dry and tight skin", 
                        "Oily skin and shine",
                        "Redness and inflammation",
                        "Uneven skin tone"
                    ],
                    "required": True
                }
            },
            "lifestyle": {
                "sleep_duration": {"type": "number", "label": "Hours of sleep per night", "min": 0, "max": 12},
                "sleep_quality": {"type": "slider", "label": "Sleep quality", "min": 1, "max": 10},
                "stress_level": {"type": "slider", "label": "Stress level", "min": 1, "max": 10},
                "water_intake": {"type": "number", "label": "Glasses of water per day", "min": 0, "max": 20}
            },
            "skin_care": {
                "skin_type": {
                    "type": "select", 
                    "label": "Skin type", 
                    "options": ["dry", "oily", "combination", "normal", "don't know"]
                }
            }
        }
    def analyze_from_bytes(self, image_bytes, user_questionnaire=None, user_id="default"):
        """Анализ из bytes с автоматическим сравнением со знаменитостями"""
        try:
            temp_path = f"temp_{user_id}_{int(time.time())}.jpg"
            
            nparr = np.frombuffer(image_bytes, np.uint8)
            img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
            cv2.imwrite(temp_path, img)
            
            result = self.analyze_3d_scan(
                front_path=temp_path,
                left_path=temp_path,
                right_path=temp_path,
                user_questionnaire=user_questionnaire,
                user_id=user_id
            )
            
            import os
            if os.path.exists(temp_path):
                os.remove(temp_path)
            
            # ДОБАВЛЯЕМ СРАВНЕНИЕ СО ЗНАМЕНИТОСТЯМИ
            if "error" not in result and self.celebrity_database:
                celebrity_match = self.get_celebrity_match(result)
                result['celebrity_match'] = celebrity_match
            
            return result
            
        except Exception as e:
            return {"error": f"Bytes analysis failed: {str(e)}"}
        
    def analyze_3d_scan(self, front_path, left_path, right_path, user_questionnaire=None, user_id="default"):
        """Main analysis method"""
        start_time = time.time()
        
        try:
            # Analyze images with preprocessing
            front_data = self._analyze_frontal(front_path)
            left_data = self._analyze_profile(left_path)
            right_data = self._analyze_profile(right_path)
            
            # Calculate scores
            scores = self._calculate_scores(front_data, left_data, right_data, user_questionnaire)
            metrics = self._compile_metrics(front_data)
            
            # Generate recommendations
            recommendations = self._generate_recommendations(scores, metrics, user_questionnaire)
            trends = self._analyze_trends(scores, user_id)
            
            processing_time = round(time.time() - start_time, 2)
            
            result = {
                'scan_id': self._generate_scan_id(),
                'user_id': user_id,
                'timestamp': datetime.now().isoformat(),
                'scores': scores,
                'metrics': metrics,
                'recommendations': recommendations,
                'trends': trends,
                'questionnaire_used': user_questionnaire is not None,
                'processing_time': processing_time
            }
            
            self.analysis_history.append(result)
            return result
            
        except Exception as e:
            return {"error": f"Analysis error: {str(e)}"}
    def get_celebrity_match(self, user_scan):
        """Находит наиболее похожую знаменитость на пользователя"""
        if not self.celebrity_database:
            return {"error": "База знаменитостей пуста"}
        
        best_match = None
        best_similarity = 0
        
        for celeb_name, celeb_data in self.celebrity_database.items():
            similarity = self._calculate_similarity(user_scan, celeb_data)
            
            if similarity > best_similarity:
                best_similarity = similarity
                best_match = celeb_data
        
        if best_match and best_similarity > 0.4:
            return {
                "celebrity": best_match["name"],
                "similarity_percent": round(best_similarity * 100, 1),
                "verdict": f"Вы похожи на {best_match['name']} на {round(best_similarity * 100, 1)}%!"
            }
        else:
            return {
                "celebrity": None,
                "verdict": "Уникальный профиль! Вы особенные! ✨"
            }

    def _calculate_similarity(self, user_scan, celeb_data):
        """Вычисляет сходство по метрикам"""
        user_metrics = user_scan["metrics"]
        celeb_metrics = celeb_data["metrics"]
        
        total_similarity = 0
        metric_count = 0
        
        for metric, celeb_value in celeb_metrics.items():
            user_value = user_metrics.get(metric, 0)
            similarity = 1 - abs(user_value - celeb_value)
            total_similarity += max(0, similarity)
            metric_count += 1
        
        return total_similarity / metric_count if metric_count > 0 else 0

    def _analyze_frontal(self, image_path):
        """Analyze frontal photo with preprocessing"""
        processed_img = self._preprocess_image(image_path)
        rgb_img = cv2.cvtColor(processed_img, cv2.COLOR_BGR2RGB)
        
        mesh_results = self.face_mesh.process(rgb_img)
        emotion_analysis = self._analyze_emotions(rgb_img) if self.use_huggingface else {}
        
        return {
            'energy': self._analyze_energy(processed_img, mesh_results),
            'skin': self._analyze_skin_quality(processed_img),
            'stress': self._analyze_stress(processed_img, mesh_results, emotion_analysis)
        }

    def get_trends_data(self, user_id: str, period_days: int = 30):
        """Возвращает данные для построения графика трендов"""
        user_history = [scan for scan in self.analysis_history 
                       if scan.get('user_id') == user_id]
        
        if not user_history:
            return {"error": "No data available"}
        
        trend_data = {
            "timestamps": [],
            "aura_scores": [],
            "energy_scores": [], 
            "skin_scores": [],
            "stress_scores": []
        }
        
        for scan in user_history[-period_days:]:
            trend_data["timestamps"].append(scan["timestamp"])
            trend_data["aura_scores"].append(scan["scores"]["aura_score"])
            trend_data["energy_scores"].append(scan["scores"]["energy_score"])
            trend_data["skin_scores"].append(scan["scores"]["skin_score"])
            trend_data["stress_scores"].append(scan["scores"]["stress_score"])
        
        return trend_data

    def get_radar_chart_data(self, scan_id: str):
        """Возвращает данные для радар-диаграммы текущего состояния"""
        scan = next((s for s in self.analysis_history 
                    if s.get('scan_id') == scan_id), None)
        
        if not scan:
            return {"error": "Scan not found"}
        
        metrics = scan["metrics"]
        
        radar_data = {
            "categories": ["Энергия", "Кожа", "Стресс", "Свежесть", "Ровность тона"],
            "values": [
                scan["scores"]["energy_score"],
                scan["scores"]["skin_score"], 
                scan["scores"]["stress_score"],
                metrics["eye_freshness"] * 100,
                metrics["skin_evenness"] * 100
            ],
            "max_value": 100
        }
        
        return radar_data

    def get_comparison_visualization_data(self, scan_id1: str, scan_id2: str):
        """Данные для визуального сравнения двух сканов"""
        scan1 = next((s for s in self.analysis_history 
                     if s.get('scan_id') == scan_id1), None)
        scan2 = next((s for s in self.analysis_history 
                     if s.get('scan_id') == scan_id2), None)
        
        if not scan1 or not scan2:
            return {"error": "One or both scans not found"}
        
        comparison_data = {
            "scan1": {
                "timestamp": scan1["timestamp"],
                "scores": scan1["scores"],
                "image_path": scan1.get("front_image_path")
            },
            "scan2": {
                "timestamp": scan2["timestamp"], 
                "scores": scan2["scores"],
                "image_path": scan2.get("front_image_path")
            },
            "improvements": {
                "aura_score": scan2["scores"]["aura_score"] - scan1["scores"]["aura_score"],
                "energy_score": scan2["scores"]["energy_score"] - scan1["scores"]["energy_score"],
                "skin_score": scan2["scores"]["skin_score"] - scan1["scores"]["skin_score"]
            }
        }
        
        return comparison_data

    def _analyze_energy(self, face_img, mesh_results):
        """Analyze energy and fatigue"""
        dark_circle_score = self._analyze_dark_circles(face_img)
        eye_freshness = self._analyze_eye_freshness(face_img)
        muscle_tone = self._analyze_muscle_tone(mesh_results)
        
        energy_score = 100 - (dark_circle_score * 40 + (1 - eye_freshness) * 40 + (1 - muscle_tone) * 20)
        
        return {
            'score': max(0, min(energy_score, 100)),
            'dark_circles': dark_circle_score,
            'eye_freshness': eye_freshness,
            'muscle_tone': muscle_tone
        }

    def _analyze_dark_circles(self, face_img):
        """Analyze dark circles under eyes"""
        # Extract eye regions using facial landmarks
        eye_regions = self._extract_eye_regions(face_img)
        
        if len(eye_regions) < 2:
            return 0.5
        
        dark_scores = []
        for eye_region in eye_regions:
            h, w = eye_region.shape[:2]
            under_eye = eye_region[int(h*0.7):h, :]
            
            if under_eye.size == 0:
                continue
            
            hsv = cv2.cvtColor(under_eye, cv2.COLOR_RGB2HSV)
            
            # Dark blue/purple tones (dark circles)
            lower_blue = np.array([100, 40, 20])
            upper_blue = np.array([140, 255, 150])
            blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)
            
            # Dark brown tones
            lower_brown = np.array([0, 40, 20])
            upper_brown = np.array([20, 255, 150])
            brown_mask = cv2.inRange(hsv, lower_brown, upper_brown)
            
            combined_mask = cv2.bitwise_or(blue_mask, brown_mask)
            dark_ratio = np.sum(combined_mask > 0) / combined_mask.size
            
            dark_scores.append(min(dark_ratio * 2, 1.0))
        
        return np.mean(dark_scores) if dark_scores else 0.0

    def _extract_eye_regions(self, face_img):
        """Extract eye regions using facial landmarks"""
        rgb_face = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
        mesh_results = self.face_mesh.process(rgb_face)
        
        if not mesh_results.multi_face_landmarks:
            return []
        
        eye_regions = []
        landmarks = mesh_results.multi_face_landmarks[0].landmark
        
        # Eye landmarks indices for MediaPipe
        left_eye_indices = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246]
        right_eye_indices = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]
        
        for eye_indices in [left_eye_indices, right_eye_indices]:
            points = []
            for idx in eye_indices:
                x = int(landmarks[idx].x * face_img.shape[1])
                y = int(landmarks[idx].y * face_img.shape[0])
                points.append([x, y])
            
            if points:
                x_coords = [p[0] for p in points]
                y_coords = [p[1] for p in points]
                x_min, x_max = min(x_coords), max(x_coords)
                y_min, y_max = min(y_coords), max(y_coords)
                
                # Add padding
                padding = 5
                x_min = max(0, x_min - padding)
                y_min = max(0, y_min - padding)
                x_max = min(face_img.shape[1], x_max + padding)
                y_max = min(face_img.shape[0], y_max + padding)
                
                eye_region = face_img[y_min:y_max, x_min:x_max]
                if eye_region.size > 0:
                    eye_regions.append(eye_region)
        
        return eye_regions

    def _analyze_eye_freshness(self, face_img):
        """Analyze eye freshness and brightness"""
        eye_regions = self._extract_eye_regions(face_img)
        
        if len(eye_regions) < 2:
            return 0.5
        
        freshness_scores = []
        for eye_region in eye_regions:
            h, w = eye_region.shape[:2]
            eye_white = eye_region[int(h*0.2):int(h*0.6), int(w*0.2):int(w*0.8)]
            
            if eye_white.size == 0:
                continue
            
            brightness = np.mean(cv2.cvtColor(eye_white, cv2.COLOR_RGB2GRAY))
            freshness_scores.append(brightness / 255.0)
        
        return np.mean(freshness_scores) if freshness_scores else 0.5

    def _analyze_muscle_tone(self, mesh_results):
        """Analyze facial muscle tone"""
        if not mesh_results.multi_face_landmarks:
            return 0.5
        return 0.7  # Placeholder

    def _analyze_skin_quality(self, face_img):
        """Analyze skin quality"""
        face_resized = cv2.resize(face_img, (200, 200))
        
        # Color evenness
        hsv = cv2.cvtColor(face_resized, cv2.COLOR_RGB2HSV)
        color_evenness = max(0, 1 - np.std(hsv[:,:,0]) / 30)
        
        # Texture smoothness
        gray = cv2.cvtColor(face_resized, cv2.COLOR_RGB2GRAY)
        smoothness = max(0, 1 - cv2.Laplacian(gray, cv2.CV_64F).var() / 500)
        
        # Skin cleanliness
        cleanliness = self._analyze_skin_cleanliness(face_resized)
        
        skin_score = (color_evenness * 0.4 + smoothness * 0.4 + cleanliness * 0.2) * 100
        
        return {
            'score': max(0, min(skin_score, 100)),
            'evenness': color_evenness,
            'smoothness': smoothness,
            'cleanliness': cleanliness
        }

    def _analyze_skin_cleanliness(self, face_img):
        """Analyze skin cleanliness (redness/inflammation)"""
        hsv = cv2.cvtColor(face_img, cv2.COLOR_RGB2HSV)
        
        # Red areas (inflammation)
        lower_red1 = np.array([0, 50, 50])
        upper_red1 = np.array([10, 255, 255])
        lower_red2 = np.array([170, 50, 50])
        upper_red2 = np.array([180, 255, 255])
        
        mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
        mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
        red_mask = mask1 + mask2
        
        red_ratio = np.sum(red_mask > 0) / red_mask.size
        
        return max(0, 1 - red_ratio * 3)

    def _analyze_stress(self, face_img, mesh_results, emotion_analysis):
        """Analyze stress indicators"""
        brow_tension = 0.3
        mouth_tension = 0.4
        asymmetry = 0.2
        
        base_stress_score = 100 - (brow_tension * 40 + mouth_tension * 35 + asymmetry * 25)
        
        if emotion_analysis:
            emotion_stress = emotion_analysis.get('computed_stress', base_stress_score)
            combined_stress = (base_stress_score * 0.7 + emotion_stress * 0.3)
            base_stress_score = max(0, min(combined_stress, 100))
        
        return {
            'score': base_stress_score,
            'brow_tension': brow_tension,
            'mouth_tension': mouth_tension,
            'asymmetry': asymmetry
        }

    def _analyze_emotions(self, img_rgb):
        """Analyze emotions with Hugging Face"""
        try:
            from PIL import Image
            pil_img = Image.fromarray(img_rgb)
            emotion_results = self.emotion_detector(pil_img)
            
            stress_indicators = ['sad', 'angry', 'fear', 'disgust']
            stress_score = 0
            for result in emotion_results:
                if result['label'].lower() in stress_indicators:
                    stress_score += result['score']
            
            return {
                'emotions': emotion_results,
                'computed_stress': min(stress_score * 100, 100),
                'dominant_emotion': emotion_results[0]['label'] if emotion_results else 'neutral'
            }
        except Exception as e:
            print(f"Emotion analysis failed: {e}")
            return {}

    def _analyze_profile(self, image_path):
        """Analyze profile photo"""
        if not image_path:
            return {}
            
        try:
            processed_img = self._preprocess_image(image_path)
            return {
                'puffiness': 0.3,  # Placeholder
                'symmetry': 0.8    # Placeholder
            }
        except:
            return {}

    def _calculate_scores(self, front_data, left_data, right_data, questionnaire):
        """Calculate final scores"""
        energy_score = front_data['energy']['score']
        skin_score = front_data['skin']['score']
        stress_score = front_data['stress']['score']
        
        # Adjust based on profile data
        if left_data and right_data:
            puffiness_avg = (left_data.get('puffiness', 0) + right_data.get('puffiness', 0)) / 2
            energy_score = energy_score * 0.9 + (100 - puffiness_avg * 100) * 0.1
        
        # Adjust based on questionnaire
        if questionnaire:
            sleep_quality = questionnaire.get('lifestyle', {}).get('sleep_quality', 5)
            energy_score = energy_score * 0.8 + (sleep_quality * 10) * 0.2
        
        aura_score = (energy_score * 0.4 + skin_score * 0.35 + stress_score * 0.25)
        
        return {
            'aura_score': round(aura_score, 1),
            'energy_score': round(energy_score, 1),
            'skin_score': round(skin_score, 1),
            'stress_score': round(stress_score, 1)
        }

    def _compile_metrics(self, front_data):
        """Compile all metrics"""
        return {
            'dark_circles': front_data['energy']['dark_circles'],
            'eye_freshness': front_data['energy']['eye_freshness'],
            'skin_smoothness': front_data['skin']['smoothness'],
            'skin_evenness': front_data['skin']['evenness'],
            'skin_cleanliness': front_data['skin']['cleanliness'],
            'brow_tension': front_data['stress']['brow_tension'],
            'mouth_tension': front_data['stress']['mouth_tension']
        }

    def _generate_recommendations(self, scores, metrics, questionnaire):
        """Generate intelligent recommendations"""
        recommendations = []
        
        # Energy recommendations
        if metrics['dark_circles'] > 0.1:
            confidence = min(metrics['dark_circles'] * 5, 1.0)
            priority = 'high' if metrics['dark_circles'] > 0.2 else 'medium'
            recommendations.append({
                'category': 'energy',
                'priority': priority,
                'message': 'Dark circles detected. Recommended: 7-8 hours sleep, cold compresses',
                'action': 'improve_sleep',
                'confidence': round(confidence, 2)
            })
        
        if metrics['eye_freshness'] < 0.5:
            recommendations.append({
                'category': 'energy',
                'priority': 'medium',
                'message': 'Tired eyes. Reduce screen time, use eye drops',
                'action': 'reduce_screen_time',
                'confidence': round(1 - metrics['eye_freshness'], 2)
            })
        
        # Skin recommendations
        if scores['skin_score'] < 60:
            recommendations.append({
                'category': 'skin',
                'priority': 'high' if scores['skin_score'] < 40 else 'medium',
                'message': 'Skin requires care. Use moisturizing products',
                'action': 'skin_hydration',
                'confidence': round(1 - (scores['skin_score'] / 100), 2)
            })
        
        # Stress recommendations
        if scores['stress_score'] < 70:
            recommendations.append({
                'category': 'stress',
                'priority': 'medium',
                'message': 'Elevated stress. Try breathing techniques',
                'action': 'stress_management',
                'confidence': round(1 - (scores['stress_score'] / 100), 2)
            })
        
        recommendations.sort(key=lambda x: (x['priority'] == 'high', x['confidence']), reverse=True)
        return recommendations

    def _analyze_trends(self, current_scores, user_id):
        """Analyze trends from history"""
        user_history = [scan for scan in self.analysis_history if scan.get('user_id') == user_id]
        
        if len(user_history) < 2:
            return ["First analysis - trends available after next scan"]
        
        previous = user_history[-1]
        trends = []
        
        for metric in ['aura_score', 'energy_score', 'skin_score', 'stress_score']:
            current = current_scores[metric]
            previous_val = previous['scores'].get(metric, current)
            change = current - previous_val
            
            if abs(change) > 2:
                direction = "improved" if change > 0 else "declined"
                trends.append(f"{metric}: {direction} by {abs(change):.1f} points")
        
        return trends if trends else ["Indicators stable"]

    def _generate_scan_id(self):
        timestamp = str(time.time())
        return f"scan_{hashlib.md5(timestamp.encode()).hexdigest()[:8]}"

    def save_results(self, result, filename="aura_analysis.json"):
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(result, f, indent=2, ensure_ascii=False)
        return filename

# Usage example
if __name__ == "__main__":
    analyzer = AuraHealthAnalyzer(use_huggingface=True)
    
    # Example questionnaire data
    questionnaire = {
        "personal_info": {"age": 28, "gender": "female"},
        "main_concerns": {"primary": ["Fatigue and lack of sleep", "Dark circles under eyes"]},
        "lifestyle": {"sleep_quality": 4, "stress_level": 7, "water_intake": 5},
        "skin_care": {"skin_type": "dry"}
    }
    
    # Analyze images
    result = analyzer.analyze_3d_scan(
        front_path="front.png",
        left_path="left.png",
        right_path="right.png",
        user_questionnaire=questionnaire
    )
    
    print(json.dumps(result, indent=2, ensure_ascii=False))
    
    # Save results
    analyzer.save_results(result)

Device set to use cpu


✅ Hugging Face models loaded successfully
{
  "scan_id": "scan_731fc057",
  "user_id": "default",
  "timestamp": "2025-10-11T21:13:56.448752",
  "scores": {
    "aura_score": 44.2,
    "energy_score": 26.2,
    "skin_score": 40.6,
    "stress_score": 78.1
  },
  "metrics": {
    "dark_circles": 1.0,
    "eye_freshness": 0.21732492997198877,
    "skin_smoothness": 0.514296673205,
    "skin_evenness": 0,
    "skin_cleanliness": 1.0,
    "brow_tension": 0.3,
    "mouth_tension": 0.4
  },
  "recommendations": [
    {
      "category": "energy",
      "priority": "high",
      "message": "Dark circles detected. Recommended: 7-8 hours sleep, cold compresses",
      "action": "improve_sleep",
      "confidence": 1.0
    },
    {
      "category": "energy",
      "priority": "medium",
      "message": "Tired eyes. Reduce screen time, use eye drops",
      "action": "reduce_screen_time",
      "confidence": 0.78
    },
    {
      "category": "skin",
      "priority": "medium",
      "message":

In [6]:
with open("aura_analysis.json", "r", encoding="utf-8") as f:
    file_content = f.read()
    print("File content:")
    print(file_content)

File content:
{
  "scan_id": "scan_011eb345",
  "user_id": "default",
  "timestamp": "2025-10-11T21:10:47.145046",
  "scores": {
    "aura_score": 44.2,
    "energy_score": 26.2,
    "skin_score": 40.6,
    "stress_score": 78.1
  },
  "metrics": {
    "dark_circles": 1.0,
    "eye_freshness": 0.21732492997198877,
    "skin_smoothness": 0.514296673205,
    "skin_evenness": 0,
    "skin_cleanliness": 1.0,
    "brow_tension": 0.3,
    "mouth_tension": 0.4
  },
  "recommendations": [
    {
      "category": "energy",
      "priority": "high",
      "message": "Dark circles detected. Recommended: 7-8 hours sleep, cold compresses",
      "action": "improve_sleep",
      "confidence": 1.0
    },
    {
      "category": "energy",
      "priority": "medium",
      "message": "Tired eyes. Reduce screen time, use eye drops",
      "action": "reduce_screen_time",
      "confidence": 0.78
    },
    {
      "category": "skin",
      "priority": "medium",
      "message": "Skin requires care. Use mo

In [9]:
# Usage example
if __name__ == "__main__":
    analyzer = AuraHealthAnalyzer(use_huggingface=True)
    
    # ТЕСТ 1: Анализ из файла как bytes (имитация бэкенда)
    print("🧪 Тестируем работу с bytes...")
    
    try:
        # Читаем файл как bytes (как будет в бэкенде)
        with open("front.png", "rb") as f:
            image_bytes = f.read()
        
        print(f"📸 Размер bytes: {len(image_bytes)} байт")
        
        # Анализируем через bytes метод
        result = analyzer.analyze_from_bytes(image_bytes)
        
        if "error" in result:
            print(f"❌ Ошибка: {result['error']}")
        else:
            print("✅ УСПЕХ! Анализ через bytes работает!")
            print(f"Aura Score: {result['scores']['aura_score']}")
            print(f"Время обработки: {result['processing_time']}сек")
            
    except FileNotFoundError:
        print("❌ Файл front.png не найден! Создай тестовый файл.")
    except Exception as e:
        print(f"❌ Общая ошибка: {e}")
    
    # ТЕСТ 2: Сравниваем со старым методом
    print("\n🧪 Сравнение с файловым методом...")
    try:
        result_old = analyzer.analyze_3d_scan(
            front_path="front.png",
            left_path="front.png", 
            right_path="front.png"
        )
        print("✅ Файловый метод тоже работает")
    except:
        print("❌ Файловый метод не работает")

Device set to use cpu


🧪 Тестируем работу с bytes...
📸 Размер bytes: 117647 байт
✅ УСПЕХ! Анализ через bytes работает!
Aura Score: 50.5
Время обработки: 2.25сек

🧪 Сравнение с файловым методом...
✅ Файловый метод тоже работает


In [22]:
# Usage example
if __name__ == "__main__":
    analyzer = AuraHealthAnalyzer(use_huggingface=True)
    
    # ТЕСТИРУЕМ С БАЙТАМИ (как в бэкенде)
    print("🧪 Тестируем анализ через bytes со знаменитостями...")
    
    try:
        # Читаем файл как bytes
        with open("front.png", "rb") as f:
            image_bytes = f.read()
        
        # Анализируем через bytes метод (должен добавить сравнение со знаменитостями)
        result = analyzer.analyze_from_bytes(
            image_bytes=image_bytes,
            user_questionnaire={
                "personal_info": {"age": 28, "gender": "female"},
                "main_concerns": {"primary": ["Fatigue and lack of sleep", "Dark circles under eyes"]},
                "lifestyle": {"sleep_quality": 4, "stress_level": 7, "water_intake": 5},
                "skin_care": {"skin_type": "dry"}
            }
        )
        
        print("🎯 РЕЗУЛЬТАТ АНАЛИЗА СО ЗНАМЕНИТОСТЯМИ:")
        print(f"Aura Score: {result['scores']['aura_score']}")
        
        # Проверяем есть ли сравнение со знаменитостями
        if 'celebrity_match' in result:
            celebrity_info = result['celebrity_match']
            print(f"🌟 {celebrity_info['verdict']}")
            print(f"📊 Сходство: {celebrity_info['similarity_percent']}%")
        else:
            print("❌ Сравнение со знаменитостями не добавлено в результат")
            
    except Exception as e:
        print(f"❌ Ошибка: {e}")

🔄 Создаем тестовую базу знаменитостей...
✅ Создана тестовая база из 5 знаменитостей


Device set to use cpu


✅ Hugging Face models loaded successfully
🧪 Тестируем анализ через bytes со знаменитостями...
🎯 РЕЗУЛЬТАТ АНАЛИЗА СО ЗНАМЕНИТОСТЯМИ:
Aura Score: 50.0
🌟 Вы похожи на Брэд Питт на 49.9%!
📊 Сходство: 49.9%
